# App Assistant
- This notebook is to replicate what we were able to achieve on the AWS console
- Creating through bedrock-agent for more confirugation and control

In [1]:
import boto3
import sys
import json
import pprint
from botocore.client import Config
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError
import os
import random
from retrying import retry
import time
from utility import *
print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.35.34


In [ ]:
# Create bedrock agent client
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name or 'ca-central-1'
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime", config=bedrock_config)

# Define FM to be used for generations 
model_id = "anthropic.claude-3-sonnet-20240229-v1:0" 
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'

# knowledge-base-app-assistant
kb_id = 'KESDTCXEJE'

In [ ]:

# R and G is to `retrieve_and_generate`
def retrieve_and_generate(query, kb_id, model_arn, max_results, prompt_template):
    response = bedrock_agent_client.retrieve_and_generate(
            input={
                'text': query
            },
        retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn,

            # Retrieval Configuration
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results,
                    'overrideSearchType': 'HYBRID' or 'SEMANTIC'
                    }
                }, 

            # Generation Configuration
            'generationConfiguration': {
                    'promptTemplate': {
                        'textPromptTemplate': prompt_template
                    } ,
                    'inferenceConfig': {
                    'textInferenceConfig': {
                        'maxTokens': 512,
                        'temperature': 1.0,
                        'topP': 1.0
                        }
                    }
                },

            # Orchestration Confirugation
            'orchestrationConfiguration': {
                'queryTransformationConfiguration': {
                    'type': 'QUERY_DECOMPOSITION'
                    }
                } 
            
            } # Knowledge base configuration
        }
    )
    return response


# Printing Generation Results refers to the citations / references from the KB
def print_generation_results(query, response, print_context = False):
    generated_text = response['output']['text']
    print (query)
    print(generated_text)
    
    if print_context is True:
        ## print out the source attribution/citations from the original documents to see if the response generated belongs to the context.
        citations = response["citations"]
        contexts = []
        for citation in citations:
            retrievedReferences = citation["retrievedReferences"]
            for reference in retrievedReferences:
                contexts.append(reference["content"]["text"])
    
        print('\n\n\nRetrieved Context:\n')
        pprint.pp(contexts)


# Test with potential questions

In [ ]:
# Assigning the default knowledge base prompt

path = '../machine-learning/4. Knowledge Base Template/'
# filename = 'RoleSwitching_V4_Oct2.md'
filename = 'App_Assistant_V1_Oct10.md'

with open(path+filename, 'r') as f:
    prompt_template = f.read()
    

In [ ]:
path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions2.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')

for i in queries:
    response = retrieve_and_generate(query = i, kb_id = kb_id, model_arn = model_arn, max_results = 10, prompt_template = prompt_template)
    print_generation_results(i, response)
    print ('---------------------------------------------------- \n')
    

In [ ]:
path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions3.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')

for i in queries:
    response = retrieve_and_generate(query = i, kb_id = kb_id, model_arn = model_arn, max_results = 10, prompt_template = prompt_template)
    print_generation_results(i, response)
    print ('---------------------------------------------------- \n')
    

In [ ]:
path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions4-Unrelated.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')

for i in queries:
    response = retrieve_and_generate(query = i, kb_id = kb_id, model_arn = model_arn, max_results = 10, prompt_template = prompt_template)
    print_generation_results(i, response)
    print ('---------------------------------------------------- \n')
    

# Using bedrock runtime NOT agent

In [9]:

# Bedrock Agent runtime
pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = 'ca-central-1'
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime", config=bedrock_config, region_name = region)

# KB ID
kb_id = 'KESDTCXEJE'


In [10]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [11]:
# Assigning the default knowledge base prompt

path = '../machine-learning/4. Knowledge Base Template/'
filename = 'App_Assistant_V1_Oct10.md'

with open(path+filename, 'r') as f:
    prompt_template = f.read()


In [22]:
# Loading the questions that we want to ask

path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions2.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')


# Going through the queries to send to LLM

for query in queries:
    print ('---------------------------------------------------- \n')

    # Acquire the context
    response = retrieve(query, kb_id)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)

    # Create the prompt
    prompt = f"""
    Human: Use the following <context> to answer the <query> according to your role in <prompt_template>. 
    <prompt_template>
    {prompt_template}
    </prompt_template>
    
    <context>
    {contexts}
    </context>
    
    <query>
    {query}
    </query>
    
    Do not respond if the prompt is not related to the app.
    
    Assistant:"""

    # Prepare the information to be invoked into the model
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(prompt_template, contexts, query)}]}]
    sonnet_payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "messages": messages,
        "temperature": 1,
        "top_p": 1
            }  )
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 
    accept = 'application/json'
    contentType = 'application/json'
    response = bedrock_client.invoke_model(body=sonnet_payload, 
                                           modelId=modelId, 
                                           accept=accept, 
                                           contentType=contentType)

    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text = response_body.get('content')[0]['text']
    # pp.pprint (query)
    # pp.pprint(response_text)
    print (query)
    print (response_text)
    

---------------------------------------------------- 

Is it possible to customize the app without logging in?
Yes, you can personalize the application without logging in to the app. The context states: "Can I personalize the application without logging in to the app? Yes, you can personalize the application without logging in to the app."
---------------------------------------------------- 

Am I required to accept the End User License Agreement?
Yes, accepting the End User License Agreement is required to use the Manage My Pain app. The context states: "After the Intro, you will be shown our End User License Agreement. This will make sure that you understand the terms and conditions of using Manage My Pain. Press "Agree" and you can start using the application."
---------------------------------------------------- 

Why isn’t my specific pain condition listed?
Our users have added thousands of conditions, but we just show the most common ones in the app. If you can't find the condit

In [21]:
# Loading the questions that we want to ask

path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions3.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')


# Going through the queries to send to LLM

for query in queries:
    print ('---------------------------------------------------- \n')

    # Acquire the context
    response = retrieve(query, kb_id)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)

    # Create the prompt
    prompt = f"""
    Human: Use the following <context> to answer the <query> according to your role in <prompt_template>. 
    <prompt_template>
    {prompt_template}
    </prompt_template>
    
    <context>
    {contexts}
    </context>
    
    <query>
    {query}
    </query>
    
    Do not respond if the prompt is not related to the app.
    
    Assistant:"""

    # Prepare the information to be invoked into the model
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(prompt_template, contexts, query)}]}]
    sonnet_payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "messages": messages,
        "temperature": 1,
        "top_p": 1
            }  )
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 
    accept = 'application/json'
    contentType = 'application/json'
    response = bedrock_client.invoke_model(body=sonnet_payload, 
                                           modelId=modelId, 
                                           accept=accept, 
                                           contentType=contentType)

    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text = response_body.get('content')[0]['text']
    # pp.pprint (query)
    # pp.pprint(response_text)
    print (query)
    print (response_text)
    

---------------------------------------------------- 

Will any of my personal information be shared?
No, your personal information will never be shared or disclosed to anyone without your explicit consent. The app's privacy policy clearly states that it protects and does not share users' personal data with third parties. Your information is securely stored and encrypted, with strict access controls to ensure your privacy is maintained. You have full control over your data and who it is shared with.
---------------------------------------------------- 

Do I need to use the same email for both my Google and Facebook accounts to link them in the app?
No, you do not need to have the same email address for your Google and Facebook account to connect them on the app. The context states: "Do I need to have the same email address for my Google and Facebook account to connect on the app? No, you do not; however, a Google and Facebook account may only be connected to a single account."

So you

In [20]:
# Loading the questions that we want to ask

path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions4-Unrelated.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')


# Going through the queries to send to LLM

for query in queries:
    print ('---------------------------------------------------- \n')

    # Acquire the context
    response = retrieve(query, kb_id)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)

    # Create the prompt
    prompt = f"""
    Human: Use the following <context> to answer the <query> according to your role in <prompt_template>. 
    <prompt_template>
    {prompt_template}
    </prompt_template>
    
    <context>
    {contexts}
    </context>
    
    <query>
    {query}
    </query>
    
    Do not respond if the prompt is not related to the app.
    
    Assistant:"""

    # Prepare the information to be invoked into the model
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(prompt_template, contexts, query)}]}]
    sonnet_payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "messages": messages,
        "temperature": 1,
        "top_p": 1
            }  )
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 
    accept = 'application/json'
    contentType = 'application/json'
    response = bedrock_client.invoke_model(body=sonnet_payload, 
                                           modelId=modelId, 
                                           accept=accept, 
                                           contentType=contentType)

    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text = response_body.get('content')[0]['text']
    # pp.pprint (query)
    # pp.pprint(response_text)
    print (query)
    print (response_text)
    

---------------------------------------------------- 

What is the weather like today?
This question is not related to the Manage My Pain app, so I do not have enough context to provide a meaningful response.
---------------------------------------------------- 

How do I start learning a new language?
The Manage My Pain app does not currently have features related to learning new languages. The app is focused on helping users track and manage their pain conditions, medications, and symptoms. If you have a question specifically about how to use the app for those purposes, I'd be happy to assist. Otherwise, I cannot provide guidance on learning new languages as it is outside the scope of this app.
---------------------------------------------------- 

What are the benefits of a healthy diet?
Do not respond to the prompt asking about the benefits of a healthy diet, as it is not related to the Manage My Pain app that I am designed to assist with.
------------------------------------------